In [35]:
%run imports.ipynb
%run utils.ipynb

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [36]:
data = pd.read_csv('bases/train.csv')

In [37]:
labels  = []

for i in data['label']:
    a = i.split(',')
    labels.append(a)

mlb = MultiLabelBinarizer(classes=("Dep", "Per", "Sup", "Usa", "Mis"))
labelsdf = pd.DataFrame(mlb.fit_transform(labels), columns=['Dep', 'Per', 'Sup', 'Usa', 'Mis']) 
data = data.assign(label_dep=labelsdf['Dep'].values, label_per=labelsdf['Per'].values, label_sup=labelsdf['Sup'].values, label_usa=labelsdf['Usa'].values, label_mis=labelsdf['Mis'].values)
data

,content_review,sentiment_analysis,domain_name,label,label_dep,label_per,label_sup,label_usa,label_mis
0,"I love the app, I have used it for years The o...",Positive,PhotosVideos,Usa,0,0,0,1,0
1,It would be better if they actually released t...,Positive,PhotosVideos,Mis,0,0,0,0,1
2,"Needs to be an option to pay monthly, or to op...",Negative,PhotosVideos,Mis,0,0,0,0,1
3,Please let there be a one week free trial I wo...,Positive,PhotosVideos,Mis,0,0,0,0,1
4,Can\'t connect to server. Failed big time. Ple...,Negative,PhotosVideos,Sup,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
5995,"I followed all the instructions airplane mode,...",Neutral,Lifestyle,Mis,0,0,0,0,1
5996,What a disappointment for 4.99 Ugh,Negative,Lifestyle,Mis,0,0,0,0,1
5997,I hate it ... this app does not work.,Negative,Lifestyle,Usa,0,0,0,1,0
5998,"Im at 28 weeks, and I was able to listen to my...",Positive,Lifestyle,Mis,0,0,0,0,1


In [38]:
from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean_text(s):
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

In [39]:
X = data['content_review'].apply(clean_text)
y = data.iloc[:, 4:8].values

In [40]:
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=42, test_size=0.2)
(X_tr.shape, y_tr.shape), (X_te.shape, y_te.shape)

(((4800,), (4800, 4)), ((1200,), (1200, 4)))

In [43]:
count_vectorizer = CountVectorizer(lowercase='true')
count_train = count_vectorizer.fit_transform(X_train) 
count_test = count_vectorizer.transform(X_test)

tfidf_vectorizer = TfidfVectorizer(use_idf = False, lowercase='true') 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 
tfidf_test = tfidf_vectorizer.transform(X_test)

In [44]:
SVC_Classifier =  OneVsRestClassifier(SVC(kernel='linear'))

#SVC_Classifier.fit(count_train, y_train)
#y_pred = SVC_Classifier.predict(count_test)

SVC_Classifier.fit(tfidf_train, y_train)
y_pred = SVC_Classifier.predict(tfidf_test)

print("Acurácia = ", accuracy_score(y_test, y_pred))

Acurácia =  0.6983333333333334


In [51]:
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier

LP = LabelPowerset(classifier=RandomForestClassifier(n_estimators=100), require_dense=[False, True])

LP.fit(count_train, y_train)
y_pred = LP.predict(count_test)

#LP.fit(tfidf_train, y_train)
#y_pred = LP.predict(tfidf_test)

print("Acurácia = ", accuracy_score(y_test, y_pred))

Acurácia =  0.6958333333333333
